In [1]:
import numpy as np
import pandas as pd
import matplotlib
import sklearn
%matplotlib inline

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [3]:
df_data = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

In [6]:
print df_data.shape

(21613, 21)


In [7]:
print df_data.columns

Index([u'id', u'date', u'price', u'bedrooms', u'bathrooms', u'sqft_living',
       u'sqft_lot', u'floors', u'waterfront', u'view', u'condition', u'grade',
       u'sqft_above', u'sqft_basement', u'yr_built', u'yr_renovated',
       u'zipcode', u'lat', u'long', u'sqft_living15', u'sqft_lot15'],
      dtype='object')


In [10]:
df_train = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
df_test = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)

In [12]:
print df_train.shape, df_test.shape

(17384, 21) (4229, 21)


In [13]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

In [14]:
sales = df_train.as_matrix()

In [16]:
sales.shape

(17384, 21)

In [29]:
my_features = sales[:,5]
my_weights = np.array([1., 1.])
predicted_value = np.dot(my_features, my_weights)
print predicted_value

ValueError: shapes (17384,) and (2,) not aligned: 17384 (dim 0) != 2 (dim 0)

In [28]:
sales[:,5]

(17384,)

In [21]:
test_predictions = predict_output(my_features, my_weights)
print test_predictions[0] # should be 1181.0
print test_predictions[1] # should be 2571.0

1180.0
1180.0
